In [1]:
!pip3 install gtts pydub langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=97256002c99cd55a9654018741acf7475e5e1a275946c4804f9c1800a2515204
  Stored in directory: /Users/danialdehvan/Library/Caches/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [3]:
from gtts import gTTS
from pydub import AudioSegment
import math

# Function to split text into chunks for approximately 5 minutes each
def split_text_into_chunks(text, max_duration_in_seconds=300):
    words = text.split(' ')
    chunks = []
    chunk = ''
    
    # Estimate the length of speech (about 120 words per minute)
    words_per_minute = 120
    words_per_second = words_per_minute / 60
    max_words_per_chunk = math.floor(max_duration_in_seconds * words_per_second)

    for word in words:
        if len(chunk.split(' ')) < max_words_per_chunk:
            chunk += word + ' '
        else:
            chunks.append(chunk.strip())
            chunk = word + ' '
    
    if chunk.strip():
        chunks.append(chunk.strip())

    return chunks

# Function to create speech files for each chunk with slower speed
def create_speech_file(text, file_index, lang='it'):
    tts = gTTS(text=text, lang=lang, slow=True)  # Set slow=True for slower voiceover
    filename = f"speech_chunk_{file_index}_slow.mp3"
    tts.save(filename)
    print(f"Created {filename} with slower speech")
    return filename

# Function to check the duration of the audio file
def get_audio_duration(filename):
    audio = AudioSegment.from_file(filename)
    return len(audio) / 1000  # Duration in seconds

# Function to add silence for empty lines (2-second delay)
def add_silence_to_audio(filename, text):
    audio = AudioSegment.from_mp3(filename)
    silent_duration = 1000  # 2 seconds of silence
    silence = AudioSegment.silent(duration=silent_duration)

    # Check for empty lines and add silence
    lines = text.split('\n')
    final_audio = AudioSegment.empty()
    
    for line in lines:
        if line.strip():  # Non-empty line
            tts = gTTS(text=line, lang='it', slow=True)  # Use slow=True for slower speech
            tts.save('temp.mp3')
            speech = AudioSegment.from_mp3('temp.mp3')
            final_audio += speech
        else:  # Empty line, add 2-second silence
            final_audio += silence

    final_audio.export(filename, format="mp3")
    print(f"Updated {filename} with silence for empty lines.")

# Read the Italian text from file
with open('sentences.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Split the text into 5-minute chunks
chunks = split_text_into_chunks(text)

# Generate speech files for each chunk and add silence for empty lines
for index, chunk in enumerate(chunks):
    audio_filename = create_speech_file(chunk, index + 1)
    
    # Add 2-second silence for empty lines in the text
    add_silence_to_audio(audio_filename, chunk)
    
    # Check the audio duration and split if needed
    duration = get_audio_duration(audio_filename)
    print(f"Duration of {audio_filename}: {duration:.2f} seconds")
    
    if duration > 300:  # If longer than 5 minutes
        print(f"{audio_filename} is longer than 5 minutes, splitting...")
        audio = AudioSegment.from_mp3(audio_filename)
        parts = math.ceil(duration / 300)
        
        for i in range(parts):
            start_time = i * 300 * 1000
            end_time = (i + 1) * 300 * 1000
            part_audio = audio[start_time:end_time]
            part_filename = f"speech_chunk_{index + 1}_part_{i + 1}.mp3"
            part_audio.export(part_filename, format="mp3")
            print(f"Created {part_filename}")

print("All speech files generated!")


Created speech_chunk_1_slow.mp3 with slower speech
Updated speech_chunk_1_slow.mp3 with silence for empty lines.
Duration of speech_chunk_1_slow.mp3: 372.48 seconds
speech_chunk_1_slow.mp3 is longer than 5 minutes, splitting...
Created speech_chunk_1_part_1.mp3
Created speech_chunk_1_part_2.mp3
Created speech_chunk_2_slow.mp3 with slower speech
Updated speech_chunk_2_slow.mp3 with silence for empty lines.
Duration of speech_chunk_2_slow.mp3: 389.38 seconds
speech_chunk_2_slow.mp3 is longer than 5 minutes, splitting...
Created speech_chunk_2_part_1.mp3
Created speech_chunk_2_part_2.mp3
Created speech_chunk_3_slow.mp3 with slower speech
Updated speech_chunk_3_slow.mp3 with silence for empty lines.
Duration of speech_chunk_3_slow.mp3: 396.71 seconds
speech_chunk_3_slow.mp3 is longer than 5 minutes, splitting...
Created speech_chunk_3_part_1.mp3
Created speech_chunk_3_part_2.mp3
Created speech_chunk_4_slow.mp3 with slower speech
Updated speech_chunk_4_slow.mp3 with silence for empty lines.